In [1]:
import PyPDF3
import re
import pandas as pd

pdf_path = '01-2023_b.pdf'

In [39]:
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

In [40]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import io
import re
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

def extract_text_from_pdf(pdf_path):
    output_string = io.StringIO()
    with open(pdf_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    return output_string.getvalue()

pdf_path = '01-2023_b.pdf'
text = extract_text_from_pdf(pdf_path)

# Dividir o texto em linhas
lines = text.split('\n')

# Remover caracteres ilegais de cada linha
lines = [ILLEGAL_CHARACTERS_RE.sub(r'', line) for line in lines]

# Salvar as linhas em um DataFrame do pandas
df = pd.DataFrame(lines, columns=['Texto'])

# Salvar o DataFrame em um arquivo Excel
df.to_excel('/Users/wilkne/Workspace/development/python_studies/src/arquivo.xlsx', index=False)

In [129]:
import pdfplumber
# import pandas as pd

pdf_path = '01-2023_b.pdf'

with pdfplumber.open(pdf_path) as pdf:
    first_page = pdf.pages[0]
    first_page = first_page.crop((40, 305, 350, 400))

im =  first_page.to_image()
im.draw_vlines([43, 140, 155, 175, 210, 232, 257, 282, 305, 319, 340], stroke='blue', stroke_width=1)

table = first_page.extract_table({
    'vertical_strategy': 'lines',
    'horizontal_strategy': 'lines',
    'intersection_x_tolerance': 10,
    'explicit_vertical_lines': [43, 140, 155, 175, 210, 232, 257, 282, 305, 319, 340],
})


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/wilkne/Workspace/development/python_studies/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/t0/9v4t04651s93690pqdrnntjr0000gn/T/ipykernel_31889/3214787.py", line 13, in <module>
    table = first_page.extract_table({
            ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wilkne/Workspace/development/python_studies/.venv/lib/python3.11/site-packages/pdfplumber/page.py", line 369, in extract_table
    table = self.find_table(tset)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wilkne/Workspace/development/python_studies/.venv/lib/python3.11/site-packages/pdfplumber/page.py", line 345, in find_table
    tables = self.find_tables(tset)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wilkne/Workspace/development/python_studies/.venv/lib/python3.11/site-packages/pdfplumber/page.py", line 339, in find_tables
    return Ta

In [132]:
import re
import pandas as pd
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def extract_text_from_pdf(pdf_path):
    output_string = StringIO()
    with open(pdf_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    return output_string.getvalue()

def extract_bill_data(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    data = []
    pattern = r'(\w[\w\s]+)\s+(\d+)\s+([\d,.-]+)\s+([\d,.-]+)\s+([\d,.-]+)\s+([\d,.-]+)%\s+([\d,.-]+)\s+([\d,.-]+)'
    matches = re.findall(pattern, text)
    for match in matches:
        item = match[0].strip()
        quantity = int(match[1])
        unit_price = float(match[2].replace(',', '.'))
        value = float(match[3].replace(',', '.'))
        pis_cofins = float(match[4].replace(',', '.'))
        icms_base = float(match[5].replace(',', '.'))
        icms_rate = float(match[6].replace('%', '')) / 100
        icms_value = float(match[7].replace(',', '.'))
        unit_rate = float(match[8].replace(',', '.'))
        data.append([item, quantity, unit_price, value, pis_cofins, icms_base, icms_rate, icms_value, unit_rate])

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data, columns=['Item', 'Quantity', 'Unit Price', 'Value', 'PIS/COFINS', 'ICMS Base', 'ICMS Rate', 'ICMS Value', 'Unit Rate'])
    return df

pdf_path = '01-2023_b.pdf'
df = extract_bill_data(pdf_path)
print(df)


Empty DataFrame
Columns: [Item, Quantity, Unit Price, Value, PIS/COFINS, ICMS Base, ICMS Rate, ICMS Value, Unit Rate]
Index: []


In [133]:
import re
import pandas as pd
from pdfminer.high_level import extract_text

def extract_bill_data(pdf_path):
    text = extract_text(pdf_path)
    data = []
    pattern = r'(\w[\w\s]+)\s+(\d+)\s+([\d,.-]+)\s+([\d,.-]+)\s+([\d,.-]+)\s+([\d,.-]+)%\s+([\d,.-]+)\s+([\d,.-]+)'
    matches = re.findall(pattern, text)
    for match in matches:
        item = match[0].strip()
        quantity = int(match[1])
        unit_price = float(match[2].replace(',', '.'))
        value = float(match[3].replace(',', '.'))
        pis_cofins = float(match[4].replace(',', '.'))
        icms_base = float(match[5].replace(',', '.'))
        icms_rate = float(match[6].replace('%', '')) / 100
        icms_value = float(match[7].replace(',', '.'))
        unit_rate = float(match[8].replace(',', '.'))
        data.append([item, quantity, unit_price, value, pis_cofins, icms_base, icms_rate, icms_value, unit_rate])

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data, columns=['Item', 'Quantity', 'Unit Price', 'Value', 'PIS/COFINS', 'ICMS Base', 'ICMS Rate', 'ICMS Value', 'Unit Rate'])
    return df

pdf_path = '01-2023_b.pdf'
df = extract_bill_data(pdf_path)
print(df)


ImportError: cannot import name 'HOCRConverter' from 'pdfminer.converter' (/Users/wilkne/Workspace/development/python_studies/.venv/lib/python3.11/site-packages/pdfminer/converter.py)